# Making the dataframe (Used in Meisler and Gabrieli, 2021)
## This must be run before other code sections
## This code contains
1) Phenotypic data loading
2) Quality Control
3) Calculating covariates and FA metrics

### If replicating this study, make sure your dataset is BIDS-compliant and:
1) Update the variable "bids_dir" to reflect where your data is stored
2) Have your phenotypic data stored in the BIDS code directory with the name "HBN_query.csv"
3) Make sure you have collected at the very least: Identifiers, Basic demographics, Clinican Diagnoses, TOWRE scores, and EHQ scores from the HBN data portal

### This could be adapted for other studies that use this suite of neuroimaging scripts on different subjects:
1) Make sure your phenotypic file is organized in the same way (one row per subject, one column per trait)
2) Either change the headings of your phenotypic file to match the HBN headers, or change the script to match your headers
3) Point the variable "HBN_query" to your phenotypic file
4) Change the "representative_subject" variable to one with good quality data and a complete gradient table

# 1) Import packages and load data
### Make sure to read the instructions in the github and the preamble to this notebook

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import filecmp
import json
import nilearn
import nilearn.image
import fsl
import fsl.wrappers
import fsl.utils.image.resample
from fsl.data.image import Image

In [ ]:
# CHANGE THIS VARIABLE TO REFLECT WHERE YOUR HBN BIDS DATA LIVE ()
bids_dir = '/om4/group/gablab/data/HBN/' # Path should end with a '/'

# IF USING YOUR OWN DATASET, CHANGE THIS VARIABLE TO A SUBJECT WITH A GOOD QUALITY RUN AND COMPLETE GRADIENT TABLE
representative_subject = 'sub-NDARAA948VFH/'
num_bad_slices_thresh = 9 # Any subjects with more bad slices than this are excluded (see QC). Change this if you wish

# TRACTS TO ANALYZE
tracts = ['AF_left','AF_right','SLF_I_left','SLF_I_right','SLF_II_left','SLF_II_right',
         'SLF_III_left','SLF_III_right','ILF_left','ILF_right','IFO_left','IFO_right',
         'UF_left','UF_right','SCP_left','SCP_right','ICP_left','ICP_right','MCP','CC_7']

# THESE VARIABLES SHOULD NOT CHANGE IF THE ANALYSIS WAS RUN ACCORDING TO INSTRUCTIONS
code_dir = bids_dir+'code/'
derivatives_dir = bids_dir+'derivatives/'
qsiprep_dir = derivatives_dir+'qsiprep/'
tractseg_dir = derivatives_dir+'TractSeg_NEW/'
freesurfer_dir = derivatives_dir+'freesurfer/'

# LOAD THE PHENOTYPIC DATA
HBN_query = pd.read_csv(code_dir+'HBN_query2.csv')

# GET SUBJECTS WITH PHENOTYPIC DATA
query_subs = ['sub-'+name[:-11] for name in HBN_query['Identifiers']]

# GET SUBJECTS WITH TractSeg SEGMENTATION DATA
tractseg_subs = np.transpose([path.split('/')[-1] for path in glob.glob(tractseg_dir+'sub-*')])
tractseg_subs.sort()

# GET SUBJECTS WITH FreeSurfer STATS DATA
freesurfer_subs = [string.split('/')[-3] for string in glob.glob(freesurfer_dir+'sub-*/stats/aseg.stats')]
freesurfer_subs.sort()

# 2) Find intersection of subjects with valid TOWRE scores, handedness data, FreeSurfer, and segmentation data

### This assumes any folder in your TractSeg directory beginning with "sub-" has complete data. Please remove failed TractSeg subjects from your TractSeg directory or rename them (e.g. "BAD_sub-XX")

In [ ]:
# GET SUBJECTS WITH VALID TOWRE SCORES, HANDEDNESS DATA
towre_query_subs = np.asarray(query_subs)[HBN_query['TOWRE,TOWRE_Valid']=='1']
hand_query_subs = np.asarray(query_subs)[HBN_query['EHQ,Data_entry']=='Complete']

# GET INTERSECTION OF SUBJECTS WITH PHENOTYPIC AND IMAGING DATA
subs_pheno_img = list(set(tractseg_subs) & set(freesurfer_subs) & set(towre_query_subs) & set(hand_query_subs))
subs_pheno_img.sort()

# FIND WHERE THESE SUBJECTS ARE LOCATED IN THE PHENOTYPIC FILE
pheno_img_query_indexes = np.transpose([np.where(np.asarray(query_subs)==sub)[0][0] for sub in subs_pheno_img])

# EXTRACT THE TOWRE SCORES
towre_scores_with_nan=HBN_query['TOWRE,TOWRE_Total_Scaled'][pheno_img_query_indexes]

# FILTER OUT THE NANs
good_inds=np.asarray(towre_scores_with_nan.isnull()==False)
towre_scores = np.asarray(list(map(int, np.asarray(towre_scores_with_nan[good_inds]))))

# GET LIST OF SUBEJCTS WITH AND FULL PHENOTYPIC DATA AND IMAGING DATA
subs_pheno_img = np.asarray(subs_pheno_img)[good_inds]

# GET THEIR LOCATIONS IN THE PHENOTYPIC FILE
pheno_img_query_indexes = pheno_img_query_indexes[good_inds]
print('A total of',np.size(subs_pheno_img),'subjects have full phenotypic and segmentation data.')

# 3) Quality Control
### Here we use the QC outputs of QSIPrep to exclude any subjects with poor quality images, inconsistent diffusion parameters, or an age > 18.
### If you have additional subjects you would like to exclude (e.g. visually analyzing QSIPrep HTML files or TractSeg outputs), add their full subject name (e.g. sub-PROJECTXXXX) to the variable "more_subs_exclude"

In [ ]:
more_subs_exclude = []

# INITIALIZE OUTPUT VARIABLES
subs_qc = [] # sub names
inds_qc = [] # phenotypic file indexes

# LOAD QSIPREP GROUP QC OUTPUT
qc_file  = qsiprep_dir+'dwiqc.json'
with open(qc_file) as f:
        qc_data = json.load(f)['subjects']
# GET ORDER OF SUBJECTS in QC Data
qc_sub_order = np.asarray([sub_data['subject_id'] for sub_data in qc_data])
# LOAD REPRESENTATIVE B-VALUES TO COMPARE AGAINST
b_val = glob.glob(qsiprep_dir+representative_subject+'/dwi/*.bval')[0]

# RUN QC
print('SUBJECTS BELOW HAVE BEEN EXCLUDED, THE THREE CRITERIA ARE LISTED NEXT TO THE SUBJECT NAME')
for ind in range(np.size(subs_pheno_img)):
    
    # EXTRACT SUBJECT NAME AND INDEX
    sub = subs_pheno_img[ind]
    query_ind = pheno_img_query_indexes[ind]
    
    # EXCLUDE SUBJECT IF MANUALLY ADDED TO EXCLUSION LIST
    if sub in more_subs_exclude:
        continue
        
    # COMPARE B-VALUES TO REPRESENTATIVE SUBJECT
    b_val_sub = glob.glob(qsiprep_dir+sub+'/dwi/*.bval')[0]
    b_val_comparison = filecmp.cmp(b_val,b_val_sub)
    
    # LOAD QC DATA AND CHECK NUMBER OF BAD SLICES
    qc_ind = [index for index in range(np.size(qc_sub_order)) if qc_sub_order[index] == sub][0]
    num_bad_slices = qc_data[qc_ind]['t1_num_bad_slices']
    
    # CHECK AGE
    age_check = np.asarray(HBN_query['Basic_Demos,Age'][query_ind]) < 19
    
    # IF QC PASSSES, SAVE OUT SUBJECT
    if num_bad_slices <= num_bad_slices_thresh and b_val_comparison and age_check:
        subs_qc.append(sub)
        inds_qc.append(pheno_img_query_indexes[ind])
    # IF NOT, PRINT SUBJECT NAME AND WHY THEY DID NOT PASS
    else: 
        print(sub, 'num_bad_slices =',num_bad_slices, ', b_val_comparison =',b_val_comparison, ', age_check =',age_check)
        
# CONVERT TO A NUMPY ARRAY FOR CONVENIENCE
subs_qc = np.asarray(subs_qc)
inds_qc = np.asarray(inds_qc)

# 4) Compute extra covariates
### These include total brain volume from FreeSurfer, as well as global FA

In [ ]:
# UNCOMMENT TO LOAD PREVIOUSLY CALCULATED VALUES
icv = np.load('icv.npy')
tbv = np.load('tbv.npy')
gFA = np.load('gFA.npy')

In [ ]:
icv = [] # Intracranial volume
tbv = [] # Total brain volume
gFA = [] # Global FA
for sub in subs_qc:
    # Load FreeSurfer Stats
    fs_stats_path = freesurfer_dir+sub+'/stats/aseg.stats'
    with open(fs_stats_path) as f:
        lines = f.readlines()
    # Extract ICV and add to list
    icv_text = lines[34]
    icv_sub = float((icv_text.split(',')[-2]))
    icv.append(icv_sub)
     # Extract total brain volume and add to list
    tbv_text = lines[13]
    tbv_sub = float((tbv_text.split(',')[-2]))
    tbv.append(tbv_sub)

    # Calculate global FA
    # define image paths  
    fa_image_path = tractseg_dir+sub+'/dwi/'+sub+'_space-reorient_desc-preproc_FA.nii.gz'
    wm_prob_path = qsiprep_dir+sub+'/anat/'+sub+'_label-WM_probseg.nii.gz'
    wm_prob_path = qsiprep_dir+sub+'/anat/'+sub+'_label-WM_probseg.nii.gz'
    wm_prob_path_reorient = tractseg_dir+sub+'/dwi/'+sub+'_space-reorient_label-WM_probseg.nii.gz'
    
    # call fslreorient2std command line from notebook to reorient white matter mask
    cmd = 'fslreorient2std '+wm_prob_path+ ' '+ wm_prob_path_reorient
    os.system(cmd)
    
    # this white matter mask file will be created
    wm_mask_path = wm_prob_path_reorient.replace('probseg','mask')

    # Binarize the probseg map to get a white matter mask, and load that image
    fsl.wrappers.fslmaths(wm_prob_path_reorient).thr(0.5).bin().run(wm_mask_path)
    wm_mask_img = Image(wm_mask_path)
    
    # Load the FA, resample it to the mask
    fa_img = Image(fa_image_path)
    fa_resampled = fsl.utils.image.resample.resampleToReference(fa_img, wm_mask_img)[0]
    
    # Find average FA within the white matter mask
    wm_mask_inds = wm_mask_img.data==1
    fa_wm_vals = fa_resampled[wm_mask_inds]
    fa_global = np.nanmean(fa_wm_vals)
    gFA.append(fa_global)
    
# Convert to numpy arrays and save out    
icv = np.array(icv)
tbv = np.array(tbv)
gFA = np.array(gFA)
np.save('icv.npy', icv)
np.save('tbv.npy', tbv)
np.save('gFA.npy', gFA)

# 5) Load demographic data only for QC subjects
### To distinguish between typical and poor readers, we use the conventional diagnostic criteria of an age-standardized TOWRE score threshold of 85

In [ ]:
# GET PHENOTYPE DATA OF QCed SUBJECTS
subs = subs_qc
sex = np.asarray(HBN_query['Basic_Demos,Sex'][inds_qc])
age = np.asarray(HBN_query['Basic_Demos,Age'][inds_qc])
hand = np.asarray(HBN_query['EHQ,EHQ_Total'][inds_qc]).astype(float)
towre = np.asarray(HBN_query['TOWRE,TOWRE_Total_Scaled'][inds_qc]).astype(float)
swe = np.asarray(HBN_query['TOWRE,TOWRE_SWE_Scaled'][inds_qc]).astype(float)
pde = np.asarray(HBN_query['TOWRE,TOWRE_PDE_Scaled'][inds_qc]).astype(float)
n = np.size(subs_qc)

# GET CLINICIAN DIAGNOSES
inds_tr = []
inds_rd = []
for index in range(n):
    ind_query = inds_qc[index]
    dxs = [HBN_query[key][ind_query] for key in HBN_query.keys() if 'DX' in key]
    dx_check = [('Impairment in Reading' in dx) for dx in dxs if type(dx)==str]
    if sum(dx_check)>0:
        inds_rd.append(True)
        inds_tr.append(False)         
    else:
        inds_tr.append(True)
        inds_rd.append(False)

inds_tr = np.asarray(inds_tr)
inds_rd = np.asarray(inds_rd)

# SPLIT INTO HIGH AND LOW T
towre_thresh = 85 # Conventional diagnostic criterion
inds_high_t = (towre >= towre_thresh)
inds_low_t = (towre < towre_thresh)

# 6) Compute average FA values in tracts
## Method 1 (used in paper) - Average within tract segmentation mask
## Method 2 - Average tract profiles derived from streamlines (e.g. across 100 nodes)

In [ ]:
# UNCOMMENT TO LOAD PREVIOUSLY CALCULATED VALUES
tract_vals = np.load('tract_vals.npy',allow_pickle=True)[()]
tract_vals2 = np.load('tract_vals2.npy',allow_pickle=True)[()]

In [ ]:
# INITIALIZE OUTPUT DICTIONARIES
tract_vals={}
tract_vals2={}
for tract in tracts:
    tract_vals[tract] = []
    tract_vals2[tract] = []

for sub in subs:
    # Load the FA (METHOD 1)
    fa_image_path = tractseg_dir+sub+'/dwi/'+sub+'_space-reorient_desc-preproc_FA.nii.gz'
    fa_vals = np.asarray(nilearn.image.load_img(fa_image_path).dataobj)
    
    # Load the FA tract profiles (METHOD 2)
    fa_tractometry_path = tractseg_dir+sub+'/FA_tractometry.csv'
    fa_tractometry_vals = pd.read_csv(fa_tractometry_path,delimiter=';')
    
    for tract in tracts:
        # METHOD 1
        # Load the tract mask
        tract_mask_path = tractseg_dir+sub+'/bundle_segmentations/'+tract+'.nii.gz'
        tract_mask_vals = np.asarray(nilearn.image.load_img(tract_mask_path).dataobj)
        # Isolate FA values within the mask
        tract_mask_inds = tract_mask_vals==1
        fa_tract = fa_vals[tract_mask_inds]
        # Calculate average tract FA
        mean_fa_tract = np.nanmean(fa_tract)
        tract_vals[tract].append(mean_fa_tract)
        
        # METHOD 2
        # Calculate average tractometry FA
        tract_vals2[tract].append(np.mean(fa_tractometry_vals[tract]))
        
np.save('tract_vals.npy',tract_vals)
np.save('tract_vals2.npy',tract_vals2)

# 7) Prepare pandas dataframe for statistical analysis

In [ ]:
df = pd.DataFrame()
df['subjects'] = subs
df['TR_RD'] = [int(i) for i in inds_rd]
df['HT_LT'] = [int(i) for i in inds_low_t] 
df['RD_TR'] = [int(not(i)) for i in inds_rd] # Create reverted group labels to make violinplots look better
df['LT_HT'] = [int(not(i)) for i in inds_low_t] # Create reverted group labels to make violinplots look better
df['AGE'] = age
df['SEX'] = sex
df['HTLTxSEX'] = df['HT_LT']*df['SEX']
df['TRRDxSEX'] = df['TR_RD']*df['SEX']
df['HAND'] = hand
df['ICV'] = icv
df['TBV'] = tbv
df['gFA'] = gFA
df['TOWRE'] = towre
df['SWE'] = swe
df['PDE'] = pde
df['all'] = 80 # Hack to make the violinplot work

for tract in tracts:
    df[tract] = tract_vals[tract]
    df[tract+'2'] = tract_vals2[tract]
    
df.to_pickle('df.pkl')

In [ ]:
sns.set(font_scale = 2)
sns.set_style("white") # Format background style
# Get dataframe with just covariates
df_covars = df[['AGE','SEX','HAND','gFA','ICV','TBV','TOWRE']]

# Calculate correlations across all pairwise columns
corr = df_covars.corr(method='spearman')

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax = .45, vmin = -.45, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True, annot_kws={"fontsize":15})
#plt.savefig('covars_corr.eps',format='eps')
plt.show()
pingouin.rcorr(df_covars,method='spearman',padjust='fdr_bh')